In [ ]:
# ----------------------
# Step 1: Import Libraries
# ----------------------
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import seaborn as sns
import pickle
import keras_tuner as kt


In [ ]:
# ----------------------
# Step 2: Preprocess Data
# ----------------------

# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizes pixel values from the range [0, 255] to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    zoom_range=0.2,
)

training_set = train_datagen.flow_from_directory(
    'archive/Train',
    target_size=(128, 128),  # Resize to 128x128 pixels (Compatibility to MobileNetV2)
    batch_size=64,
    class_mode='sparse',  # Sparse categorical labels, returns labels as integers
    color_mode='rgb'  # RGB input because MobileNetV2 expects 3-channel RGB input
)

class_indices = training_set.class_indices  # Get the class label-to-index mapping
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(class_indices, f)  # Save the mapping to a file

print("Class indices saved successfully:", class_indices)


In [ ]:
# Map integer class labels back to class names
reverse_class_indices = {v: k for k, v in training_set.class_indices.items()}
class_names = [reverse_class_indices[label] for label in training_set.classes]

# Create a count plot
sns.countplot(x=class_names, order=sorted(reverse_class_indices.values()))
plt.title("Class Distribution in Training Data")
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.xticks(rotation=90)  # Rotate class names for better visibility
plt.show()

In [ ]:
# Test Data Preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'archive/Test',
    target_size=(128, 128),  # Match training size
    batch_size=64,
    class_mode='sparse',     # Sparse categorical labels
    color_mode='rgb'
)

In [ ]:
# Display some augmented images
x_batch, y_batch = next(training_set)  # Get a batch of training data
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_batch[i])  # Display image in its original color format (RGB)
    
    # Get the class name using the integer label
    class_name = reverse_class_indices[y_batch[i]]
    
    plt.title(f'Class: {class_name}')  # Show class name instead of integer label
    plt.axis('off')  # Turn off the axis for a cleaner visualization

plt.tight_layout()
plt.show()


In [ ]:
# ----------------------
# Step 3: Load Pretrained Model (Feature Extractor)
# ----------------------

# Load MobileNetV2 pretrained on ImageNet
base_model = MobileNetV2(
    input_shape=(128, 128, 3),  # MobileNetV2 expects 3-channel RGB input
    include_top=False,          # Exclude the classification head, Excludes dense layer, acts as feature extractor
    weights='imagenet'          # Use pretrained weights
)

# Freeze the base model's layers to prevent training
base_model.trainable = False


In [ ]:
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)
'''
Dynamically reduces the learning rate when the validation loss plateaus (does not improve for patience epochs).
Helps the model converge better in later epochs by fine-tuning weights more cautiously.
'''

reduce_lr = ReduceLROnPlateau( # Dynamic adjustment of the learning rate during training can lead to better convergence.
    monitor='val_loss',
    factor=0.5,   # Reduce the learning rate by half
    patience=3,   # If no improvement for 3 epochs
    min_lr=1e-6   # Set a minimum learning rate
)


In [ ]:
# ----------------------
# Step 4: Build the Model
# ----------------------

# Add custom layers on top of the pretrained base model
# Define a function to build the model
def build_model(hp):
    model = Sequential([
        base_model,  # Pretrained MobileNetV2 as the base model
        GlobalAveragePooling2D(),

        # Tune the number of units in the first Dense layer
        Dense(units=hp.Int('units_1', min_value=128, max_value=512, step=64), activation='relu'),
        BatchNormalization(),
        Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)),

        # Tune the number of units in the second Dense layer
        Dense(units=hp.Int('units_2', min_value=64, max_value=256, step=64), activation='relu'),
        Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)),

        # Final output layer
        Dense(24, activation='softmax')
    ])

    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Define the tuner for hyperparameter search using the Hyperband algorithm
tuner = kt.Hyperband(
    build_model,          # The function that builds the model with tunable hyperparameters
    objective='val_accuracy',  # Optimize the validation accuracy during tuning
    max_epochs=3,         # Maximum number of epochs any model can be trained for
    factor=3,             # The reduction factor; in each round, only 1/factor models are retained
    directory='hyperparameter_tuning',  # Directory to store tuning results
    project_name='cnn_asl'              # Name of the project for organizing tuning outputs
)



In [ ]:
# Run the tuner
tuner.search(training_set, validation_data=test_set, callbacks=[model_checkpoint
])

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] # num_trials retrieves the single best parameter
best_model = tuner.get_best_models(num_models=1)[0]

print(f"Best number of units in the first dense layer: {best_hps.get('units_1')}")
print(f"Best number of units in the second dense layer: {best_hps.get('units_2')}")
print(f"Best dropout rate for first layer: {best_hps.get('dropout_1')}")
print(f"Best dropout rate for second layer: {best_hps.get('dropout_2')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

In [ ]:
# Inspect a batch of data from the generator
# To confirm shape and format 
x_batch, y_batch = next(training_set)
print("Input batch shape (x_batch):", x_batch.shape)
print("Label batch shape (y_batch):", y_batch.shape)
training_set.reset()
test_set.reset()

y_batch = np.argmax(y_batch, axis=-1)


In [ ]:
# ----------------------
# Step 6: Train the Model
# ----------------------

# Train the model
history = best_model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
    # class_weight = class_weights,
    callbacks=[reduce_lr, model_checkpoint]
)

In [ ]:
# ----------------------
# Step 7: Fine Tuning
# ----------------------


# Unfreeze the base model for fine-tuning
base_model.trainable = True


# Unfreezing deeper layers since they capture high-level features relevant
for layer in base_model.layers[:-50]:  # Freeze all layers except the last 50
    layer.trainable = False

In [ ]:
# Recompiling again after changing trainable attributes
best_model.compile(
    optimizer=Adam(learning_rate=0.0001),  # incerease LR since  frozen layers have already stabilized.
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
# Fine-tune the model
history_finetune = best_model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,  # Train for a few more epochs
    callbacks=[model_checkpoint]  # Apply early stopping for best model
)


In [ ]:
X_train, y_batch = next(training_set)  # Fetch both images and labels
X_train = X_train.reshape(len(X_train), -1)  # Flatten images
print("Shape of X_train:", X_train.shape)
print("Shape of y_batch:", y_batch.shape)  # Should be (64,)


In [ ]:
# # ----------------------
# # Step 5: Model Selection
# # ----------------------


# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, classification_report
# import numpy as np

# # Extract entire dataset from the generator
# X_train_list = []
# y_train_list = []

# # Iterate through the generator to get all batches
# for _ in range(len(training_set)):  # Number of steps per epoch
#     X_batch, y_batch = next(training_set)
#     X_train_list.append(X_batch.reshape(len(X_batch), -1))  # Flatten images
#     y_train_list.append(y_batch)

# # Combine all batches into one dataset
# X_train = np.vstack(X_train_list)  # Combine all image batches
# y_train = np.hstack(y_train_list)  # Combine all label batches

# # Check shapes
# print("Shape of X_train:", X_train.shape)  # Should be (total_samples, number_of_features)
# print("Shape of y_train:", y_train.shape)  # Should be (total_samples,)

# # Define Logistic Regression parameters for GridSearchCV
# param_grid_lr = {
#     'C': [0.1, 1, 10],
#     'solver': ['liblinear', 'lbfgs'],
#     'max_iter': [100, 200, 500]
# }

# # Train Logistic Regression with GridSearchCV
# lr_model = LogisticRegression()
# grid_search_lr = GridSearchCV(estimator=lr_model, param_grid=param_grid_lr, cv=3, scoring='accuracy', verbose=1)
# grid_search_lr.fit(X_train, y_train)  # Train on the full dataset

# # Display best parameters and training accuracy
# print("Best Logistic Regression Parameters:", grid_search_lr.best_params_)
# print("Best Logistic Regression Accuracy:", grid_search_lr.best_score_)

# # Test the model
# X_test, y_test = next(test_set)  # Fetch a batch of test data
# X_test = X_test.reshape(len(X_test), -1)  # Flatten test images
# y_pred_lr = grid_search_lr.best_estimator_.predict(X_test)

# # Evaluate model performance
# print("Logistic Regression Test Accuracy:", accuracy_score(y_test, y_pred_lr))
# print(classification_report(y_test, y_pred_lr))


In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, classification_report
# import numpy as np

# # Extract entire dataset from the generator
# X_train_list = []
# y_train_list = []

# # Iterate through the generator to get all batches
# for _ in range(len(training_set)):  # Number of steps per epoch
#     X_batch, y_batch = next(training_set)
#     X_train_list.append(X_batch.reshape(len(X_batch), -1))  # Flatten images
#     y_train_list.append(y_batch)

# # Combine all batches into one dataset
# X_train = np.vstack(X_train_list)  # Combine all image batches
# y_train = np.hstack(y_train_list)  # Combine all label batches

# # Check shapes
# print("Shape of X_train:", X_train.shape)  # Should be (total_samples, number_of_features)
# print("Shape of y_train:", y_train.shape)  # Should be (total_samples,)

# # Define hyperparameters for GridSearchCV
# param_grid_dt = {
#     'criterion': ['gini', 'entropy'],       # Splitting criteria
#     'max_depth': [10, 20, 30, None],        # Maximum depth of the tree
#     'min_samples_split': [2, 5, 10],        # Minimum samples required to split
#     'min_samples_leaf': [1, 2, 4]           # Minimum samples per leaf node
# }

# # Initialize Decision Tree model
# dt_model = DecisionTreeClassifier(random_state=42)

# # GridSearchCV to find the best parameters
# grid_search_dt = GridSearchCV(estimator=dt_model, param_grid=param_grid_dt, cv=3, scoring='accuracy', verbose=1)
# grid_search_dt.fit(X_train, y_train)  # Train on the entire dataset

# # Display best parameters and training accuracy
# print("Best Decision Tree Parameters:", grid_search_dt.best_params_)
# print("Best Decision Tree Accuracy:", grid_search_dt.best_score_)

# # Test the model
# X_test_list = []
# y_test_list = []

# # Extract entire test dataset
# for _ in range(len(test_set)):
#     X_batch, y_batch = next(test_set)
#     X_test_list.append(X_batch.reshape(len(X_batch), -1))  # Flatten test images
#     y_test_list.append(y_batch)

# # Combine test batches
# X_test = np.vstack(X_test_list)
# y_test = np.hstack(y_test_list)

# # Make predictions on the test set
# dt_best_model = grid_search_dt.best_estimator_
# y_pred_dt = dt_best_model.predict(X_test)

# # Evaluate model performance
# print("Decision Tree Test Accuracy:", accuracy_score(y_test, y_pred_dt))
# print(classification_report(y_test, y_pred_dt))

In [ ]:

# -------------------------------
# Predict on Entire Test Dataset
# -------------------------------

# Initialize arrays for true labels and predictions
true_labels = []
predicted_labels = []

# Loop through the test dataset in batches
for images, labels in test_set:
    predictions = best_model.predict(images)  # Get predictions
    predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class indices
    true_labels.extend(labels)  # Store true labels
    predicted_labels.extend(predicted_classes)  # Store predicted labels

    # Break after processing all test data (important for generators)
    if len(true_labels) >= test_set.samples:
        break

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# -------------------------------
# 1. Confusion Matrix
# -------------------------------

# Compute and display confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(training_set.class_indices.keys()))

# Adjust figure size before plotting
plt.figure(figsize=(10,10))  # Adjust the width and height as needed
disp.plot(cmap='viridis', xticks_rotation='vertical', values_format='d')

# Add title and show plot
plt.title('Confusion Matrix', fontsize=20)
plt.xlabel('Predicted Labels', fontsize=14)
plt.ylabel('True Labels', fontsize=14)
plt.show()

# -------------------------------
# 2. Classification Report
# -------------------------------
# Generate classification metrics
print("Classification Report:\n")
print(classification_report(true_labels, predicted_labels, target_names=list(training_set.class_indices.keys())))



In [ ]:


# Display predictions on test images
plt.figure(figsize=(10, 10))
test_images, test_labels = next(test_set)  # Get a batch of test data
predictions = best_model.predict(test_images)

for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_images[i])  # Display in original color format (e.g., RGB)

    # Get predicted and true labels as class names
    predicted_index = np.argmax(predictions[i])
    predicted_label = reverse_class_indices[predicted_index]
    true_label = reverse_class_indices[test_labels[i]]

    plt.title(f'Pred: {predicted_label}, True: {true_label}')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt


history_finetune_dict = history_finetune.history

# Extract accuracy and loss for training and validation
fine_tune_train_acc = history_finetune_dict['accuracy']
fine_tune_val_acc = history_finetune_dict['val_accuracy']
fine_tune_train_loss = history_finetune_dict['loss']
fine_tune_val_loss = history_finetune_dict['val_loss']

# Define the number of epochs
fine_tune_epochs = range(1, len(fine_tune_train_acc) + 1)

# Set figure size and style
plt.figure(figsize=(14, 6))

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(fine_tune_epochs, fine_tune_train_acc, label='Training Accuracy (Fine-tuning)', color='blue')
plt.plot(fine_tune_epochs, fine_tune_val_acc, label='Validation Accuracy (Fine-tuning)', color='orange')
plt.title('Fine-Tuning: Training and Validation Accuracy', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)

# Plot Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(fine_tune_epochs, fine_tune_train_loss, label='Training Loss (Fine-tuning)', color='blue')
plt.plot(fine_tune_epochs, fine_tune_val_loss, label='Validation Loss (Fine-tuning)', color='orange')
plt.title('Fine-Tuning: Training and Validation Loss', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)

# Show plots
plt.tight_layout()
plt.show()


In [ ]:
# Save CNN model
best_model.save('cnn_model.h5')

# # Save Logistic Regression model
# with open('logistic_regression_model.pkl', 'wb') as f:
#     pickle.dump(grid_search_lr.best_estimator_, f)

# # Save Decision Tree model
# with open('decision_tree_model.pkl', 'wb') as f:
#     pickle.dump(grid_search_dt.best_estimator_, f)


In [ ]:
# ----------------------
# Step 11: Predict on Outside Data
# ----------------------
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_outside_image(image_path, model, class_indices):
    # Step 1: Load and preprocess the image
    test_image = image.load_img(image_path, target_size=(128, 128), color_mode='rgb')  # Resize to match model input
    test_image = image.img_to_array(test_image) / 255.0  # Normalize to [0, 1]
    test_image = np.expand_dims(test_image, axis=0)      # Add batch dimension

    # Step 2: Predict
    result = model.predict(test_image)
    predicted_class_index = np.argmax(result)

    # Step 3: Map prediction back to class label
    reverse_class_indices = {v: k for k, v in class_indices.items()}
    predicted_label = reverse_class_indices[predicted_class_index]

    # Step 4: Combine class labels and probabilities, then sort and select top 5
    probabilities = {reverse_class_indices[i]: prob for i, prob in enumerate(result[0])}
    top_5_predictions = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)[:5]

    # Step 5: Print the predicted label and top 5 class probabilities
    print(f"Predicted Label: {predicted_label}")
    print("Top 5 Class Probabilities:")
    for label, prob in top_5_predictions:
        print(f"  {label}: {prob:.2f}")

# Example usage
predict_outside_image('archive/Prediction/L.jpg', best_model, training_set.class_indices)

